## 定义 Inception
![](/Users/poodar/Downloads/inception_block.svg)

In [2]:
# Inception 是一个含有四个并行卷基层的块，其给予 Network in Network 做了很大改进
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import nd

In [4]:
class Inception(nn.Block):
    def __init__(self, n1_1, n2_1, n2_3, n3_1, n3_5, n4_1, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # path1 1x1 convolution
        self.p1_conv_1 = nn.Conv2D(n1_1, kernel_size=1, activation='relu')
        # path2 1x1 convolution + 3x3 convolution
        self.p2_conv_1 = nn.Conv2D(n2_1, kernel_size=1, activation='relu')
        self.p2_conv_3 = nn.Conv2D(n2_3, kernel_size=3, padding=1, activation='relu') # padding = 1 使得输出的长宽和输入一致
        # path3 1x1 convolution + 5x5 convolution
        self.p3_conv_1 = nn.Conv2D(n3_1, kernel_size=1, activation='relu')
        self.p3_conv_5 = nn.Conv2D(n3_5, kernel_size=5, padding=2, activation='relu')
        # path4 3x3 MaxPooling  + 1x1 convolution
        self.p4_pool_3 = nn.MaxPool2D(pool_size=3, padding=1, strides=1)
        self.p4_conv_1 = nn.Conv2D(n4_1, kernel_size=1, activation='relu')

    def forward(self, X):
        p1 = self.p1_conv_1(X)
        p2 = self.p2_conv_3(self.p2_conv_1(X))
        p3 = self.p3_conv_5(self.p3_conv_1(X))
        p4 = self.p4_conv_1(self.p4_pool_3(X))
        return nd.concat(p1, p2, p3, p4, dim=1)

In [8]:
# test
incp = Inception(64, 96, 128, 16, 32, 32)
incp.collect_params().initialize()
X = nd.random.uniform(shape=(32, 3, 64, 64))

In [9]:
incp(X).shape

(32L, 256L, 64L, 64L)